# Using PowerShell with Intersight

If you are familiar or proficient with PowerShell or PowerShell Core, you can take advantage of the Intersight module available for download from the [PowerShell Gallery](https://www.powershellgallery.com/). Enter the word `Intersight` in `Search PowerShell packages:` and click the resulting link named `Intersight.PowerShell`.

![Intersight module returned by search](images/powershell_search_result_intersight.png)

Notice the `Install Module -Name Intersight.PowerShell` command. That's the command used to install the module in your PowerShell environment.

> Be sure you are in PowerShell itself before attempting to install the `Intersight.PowerShell` module. In PowerShell Core, the command to start PowerShell is `pwsh`.  You can check the current version of PowerShell Core with the following:

In [ ]:
pwsh --version

## Installation

Now install the Intersight module.  -Force is used to accept all prompts including any related to Intersight being an untrusted publisher.

In [ ]:
Install-Module -Name Intersight.PowerShell -Force

## Connect/Query/Filter with the PowerShell Intersight Module

Now that the Intersight PowerShell module is installed, let's use it to connect to our Intersight environment. But first, you'll need credentials, such as your private key, from your Intersight account.

### API Keys and User Authentication

To generate an Intersight API Key ID and secret (private) key from your Intersight account, login at https://intersight.com and go to the Settings menu then select API Keys and Generate API Key

![Generate API Key](images/generate_api_key.png)

A version 3 key can be used with the PowerShell and is recommended for long term compatibility with Intersight's API.

![Version 3 key](images/version_3_key.png)

The Generated API Key ID can be copied to the clipboard and used in API authentication or can be placed into a file and used with `Get-Content`.  The example below uses a file for lookup so the ApiKeyId is not hardcoded in this notebook.

Be sure to save the Secret (private) Key to a local file that only you have access to (a download option is provided by Intersight).

![Download Secret Key](images/secret_key_download.png)

To use your API keys, edit the ApiKeyId and ApiKeyFilePath lines below with your API Key ID and SecretKey.txt file location.

> Note that for Cisco Live workshops, the lines below are already correct for API keys configured on the workstations.

In [ ]:
$onprem = @{
    BasePath = "https://intersight.com"
    # ApiKeyId = "596cc79e5d91b400010d15ad/60ede8d07564612d3335edac/623354c07564612d33d9068d"
    ApiKeyId = Get-Content('./ApiKeyIdFile')
    ApiKeyFilePath = Resolve-Path('./SecretKey.txt')
    HttpSigningHeader =  @("(request-target)", "Host", "Date", "Digest")
}

#### Set Intersight configuration

Now set the Intersight configuration to use your API keys with `Set-IntersightConfiguration @onprem` where `@onprem` contains the hash set above.  Intersight's configuration will be used in subsequent API calls.

In [ ]:
Set-IntersightConfiguration @onprem

### See available commands

Now that you have API authentication configured, you can explore the list of available commands with `Get-Command -Module Intersight.Powershell`. There are many commands and to see what each one does along with any dependencies, visit [CiscoDevNet/intersight-powershell in the docs directory](https://github.com/CiscoDevNet/intersight-powershell/tree/master/docs) on GitHub.  Below we will select compute related commands that we'll use later in this notebook.

In [ ]:
Get-Command -Module Intersight.Powershell | Select-String compute

### Get-Help with cmdlets to list the physical compute inventory

The command `Get-IntersightComputePhysicalSummary` returns all the physical compute devices claimed in Intersight along with details about each one.  Before running the cmdlet, you can see help on the command including examples of usage (use the command line argument -Examples to see examples):


In [ ]:
Get-Help Get-IntersightComputePhysicalSummary

### Query the API for specific data

Intersight provides a query language to filter, select, and perform many other operations on API resources.  The query language allows the user to get only the data they want from the API without the need to post process results.  This can be very useful for API responses that possibly return a large amount of data like `Get-IntersightComputePhysicalSummary`.  Details on the query language are available at https://intersight.com/apidocs/introduction/query/#filter-query-option-filtering-the-resources and the Get-Help command showed how to perform queries using PowerShell:

`Get-IntersightComputePhysicalSummary [-At <string>] [-Tags <string>] [-Orderby <string>] [-Skip <integer>] [-Top <integer>] [-Filter <string>] [-Count <boolean>] [-Expand <string>] [-Inlinecount <string>] [-Select <string>] [-Apply <string>] [<CommonParameters>]`

#### Getting the number of physical compute devices claimed in Intersight

The API has limits on the number of resources returned from any call, so there are count options that can be used to determine the total number of matching resources.  The `-Count` option will return a count of matching resources in the API (not the resources themselves) and `-InlineCount` will return both the count and the resources.  Below we will get a count of resources.

In [ ]:
Get-IntersightComputePhysicalSummary -Count $true


Now will we get the count and print select fields from the returned resources.

In [ ]:
$results = Get-IntersightComputePhysicalSummary -InlineCount allpages
if ($results.Count -gt 0) {
    Write-Host "Count of compute resources is" $results.Count
    foreach ($res in $results.Results) {
        $res | Select-Object Name, Serial, Model
    }
}

### Get a filtered list of of physical compute devices claimed in Intersight

We'll finish our query examples by using `-Filter` to get only the compute resources will Model numbers that contain the string 'B200'.

In [ ]:
$results = Get-IntersightComputePhysicalSummary -Filter "contains(Model, B200)"
foreach ($res in $results.Results) {
    $res | Select-Object Name, Serial, Model
}


## Configure Resources

We've walked through retrieving data with Get cmdlets, and now we'll look at configuring resources with cmdlets to create resources (`New-`), update (`Set-`), and delete (`Remove-`).  We'll do this for a Server Profile, so we'll start by looking at Server Profile cmdlets.

In [ ]:
Get-Command -Module Intersight.PowerShell | Select-String ServerProfile

### Create a new Server Profile

New- cmdlets will be used to create resources.  You can see usages and examples of creating a Server Profile with Get-Help 

In [ ]:
Get-Help New-IntersightServerProfile

In this example we'll be creating a Server Profile in the Demo-DevNet organization of the Interisght account, so we'll get the object for that Organization and use it will the new profile.  We'll also get the properties of each object we create to see information about the Organization being used and Server Profile that is created.

In [ ]:
($org = Get-IntersightOrganizationOrganization -Name Demo-DevNet) | Get-Member -MemberType Property

In [ ]:
($prof = New-IntersightServerProfile -Name CLiveDevNet -Organization $org) | Get-Member -MemberType Property

### Update a Server Profile

We have a new Server Profile created in Intersight and represented by the $prof variable in PowerShell.  We can update the Server Profile in Intersight with the `Set-IntersightServerProfile` cmdlet.  Below we add Tags and a Description to the Server Profile we created above.  Note that `Initialize-` cmdlets can be used to create objects like Tags to be used in later operations.

In [ ]:
($tags = Initialize-IntersightMoTag -Key "Location" -Value  "Vegas") | Write-Host

In [ ]:
$prof = ($prof | Set-IntersightServerProfile -Description 'Vegas' -Tags @($tags))
Write-Host "Tags:" $prof.Tags "Description:" $prof.Description

### Delete a Server Profile

As a final example of managing Server Profiles with PowerShell, we will delete the profile created and updated above with the `Remove-IntersightServerProfile` cmdlet.

In [ ]:
($prof | Remove-IntersightServerProfile)
Write-Host $?

You've now completed an overview of the Cisco Intersight module for PowerShell.  Be sure to visit https://developer.cisco.com/site/intersight/ for additional information on automating with Intersight.